In [ ]:
import heapq
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from bidict import bidict
from collections import defaultdict

from jpeg123 import *

plt.rcParams["font.size"] = 14

# Huffman Codes

Invented in 1952 by David Huffman, a graduate student at MIT. They are one type of optimal prefix-free codes used for lossless compression. Huffman trees are constructed using the entropy of each "word" that we want to be able to transmit.

## Entropy
### What is entropy?
$$H(p) = -p\log_2(p)$$
$$H(\mathcal{D}(x)) = \sum_{x\in \mathcal{X}} -p(x) \log_2(p(x))$$
* Intuition: higher entropy means less knowledge about the distribution.
* Intuition: higher entropy of a symbol means you learned more when you received it

Which binomial distribution $\mathcal{B}(p)$ has higher entropy, 
$$\mathcal{B}(0.5): P(0)=P(1)=0.5$$ 
or 
$$\mathcal{B}(0.1): P(1)=0.1,\, P(0)=0.9?$$

$$H(\mathcal{B}(0.5)) = -0.5\log_2(0.5)\cdot 2 = 1$$
$$H(\mathcal{B}(0.1)) = -0.1\log_2(0.1) + -0.9\log_2(0.9) = 0.469\ldots$$

What is $H(\mathcal{B}(0.9))$?

In [ ]:
def binomial(p):
    """Simple binomial distribution probabilities"""
    return np.array([p, 1-p])

def Hp(p):
    """Entropy of a probability p"""
    return -p * np.log2(p)

def Hdist(probs):
    """
    Entropy of a probability distribution
    probs: np.array of sample probabilities
    """
    return sum(Hp(probs))

In [ ]:
Hdist(binomial(0.5))

In [ ]:
Hdist(binomial(0.9))

In [ ]:
probs = np.arange(0.01, 1, 0.01)
entropies = [Hdist(binomial(p)) for p in probs]
fig = plt.figure(figsize=(12,7))
plt.plot(probs, entropies)
plt.xlabel("p")
plt.ylabel("H(B(p))");

### Why do we care about the entropy of a distribution?
It turns out the entropy of a distribution represents the **minimum** bits per value it takes to send a message from the distribution on average. _Achieving_ that minimum can be very tricky.

Imagine we have a message we want to send, written in English. In this message, each word occurs with some frequency, and further each letter also occurs with some frequency.

Since the message is written in English, we have some _a priori_ knowledge about which words exist and how often they occur, and the same for letters.

There's an XKCD for that: https://xkcd.com/simplewriter/ and https://xkcd.com/1133

In [ ]:
message1 = "the quick brown fox jumps over the lazy dog"
message2 = "a baby bat beats the short cat to the table"

In [ ]:
# https://web.archive.org/web/20170918020907/http://www.data-compression.com/english.html
letters = list(map(lambda x: chr(x), list(ord("A") + np.arange(26)) + [ord(" ")]))
freqs = [0.0651738, 0.0124248, 0.0217339, 0.0349835, 0.1041442, 0.0197881, 0.0158610, 0.0492888, 0.0558094, 0.0009033, 0.0050529, 0.0331490, 0.0202124, 0.0564513, 0.0596302, 0.0137645, 0.0008606, 0.0497563, 0.0515760, 0.0729357, 0.0225134, 0.0082903, 0.0171272, 0.0013692, 0.0145984, 0.0007836, 0.1918182]
letter_freqs = {l: f for l, f in zip(letters, freqs)}

In [ ]:
fig = plt.figure(figsize=(12,7))
plt.bar(letter_freqs.keys(), letter_freqs.values())
plt.xlabel("Letter")
plt.ylabel("Relative Frequency (Q=1)");

If we want to send our message using the smallest amount of bits possible, we have many options for representing the message.


#### Custom fixed-size codes
First we could assign every English word a number, then transmit the sequence of numbers. There are ~171,000 English words, so we need at least 18 bits per number to represent every word ($2^{18} = 262,144 > 171,000$).

There are 9 words in the first message, so we need **162** bits to send the message with this encoding.

There are 10 words in the second, so we need **180** bits.

In [ ]:
print(2 ** 18)
print(len(message1.split()) * 18)
print(len(message2.split()) * 18)

Alternatively, we could use the 8-bit ASCII representation of each letter (plus space).

This results in **344** bits for both messages, since they are the same length. It turns out this is worse than enumerating every english word!

In [ ]:
print(len(message1) * 8)
print(len(message2) * 8)

We can do better though, since we only care about the 26 lower case letters and space. This requires 5 bits per character, resulting in **215** bits per message.

In [ ]:
print(len(message1) * 5)

Is this the best we can do though? The entropy of English letters is 4.07, so in theory we should only need just over 4 bits per letter. This would be **176** bits per message on average with a good compression scheme.

##### Food for thought:
Why isn't this lower than our minimum when enumerating every word?

What other information about letters could we take advantage of?

With only what we know now, how can we improve our per-letter encoding scheme?

In [ ]:
letter_probs = np.array(list(letter_freqs.values()))
letter_entropy = Hdist(letter_probs)
print("English letter entropy:", letter_entropy)
print("Average bits per message:", letter_entropy * len(message1))


## Prefix-free codes


What if we allow the encoded version of every letter, or codeword, to vary in length? We can use shorter codewords for common letters and longer codewords for rarer letters.

In order to uniquely decode every encoded message we need a way to distinguish codeword boundaries. One way to accomplish this is prefix-free coding, in which no codeword can be the prefix of any other. A natural representation for these codes is a binary tree. Each left branch adds a 0 to the codeword and each right branch a 1. Tracing branches from the root to the leaf representing your desired symbol automatically generates your codeword for that symbol.

## Huffman's Code Construction Algorithm
###### Adapted from https://math.mit.edu/~goemans/18310S15/huffman-notes.pdf

Let's illustrate the construction of a Huffman code with a small alphabet:
* **a**, $p(a)=0.4$
* **b**, $p(b)=0.05$
* **c**, $p(c)=0.18$
* **d**, $p(d)=0.07$
* **e**, $p(e)=0.1$

First, assume that d will be assigned the longest codeword because it has the lowest probability.

_Argue that if another letter has a longer codeword, we can swap with d and have better compression on average._

It follows that b will share the longest codeword length with d. Now we know our prefix code's tree representation has the subtree:

![subtree bd](./bdtree.png)

Now we can replace the b-d subtree with a placeholder letter $\alpha$, which has associated probability 
$$p(\alpha) = p(b) + p(d) = 0.12.$$

Next, we repeat the reasoning from the previous step to construct a new subtree with the next two lowest probability letters, f ($p=0.1$) and $\alpha$ ($p=0.12$).

![falpha](falphatree.png)

Expanding $\alpha$, we get the partial prefix code

![fbd](fbdtree.png)

Aggregating f and $\alpha$ into $\beta$, $p(\beta)=0.22$ we continue this reverse tree construction. 

The next two letters to be chosen are actually c and e, creating $\gamma$, $p(\gamma)=0.38$.

We choose $\beta$ and $\gamma$ to create $\delta$, $p(\delta)=0.6$. Now we are left with only a and $\delta$, and we have finally reached the root of the code tree.

![adelta](adeltatree.png)

We expand the each intermediate letter in the tree to achieve our final code.
![fulltree](fulltree.png)
![codewords](codewords.png)

Are there any prefix codewords?

What is the expected number of bits per letter, and how does this compare to the entropy of our alphabet?
$$\mathbb{E}[\text{bits/letter}] = \sum_l p(l)\cdot \text{len}(l)$$

In [ ]:
expected_bits = 1 * 0.4 + 4 * 0.05 + 3 * 0.18 + 4 * 0.07 + 3 * 0.2 + 3 * 0.1
print("E[b/l] =", expected_bits)

In [ ]:
entropy_alphabet = Hdist(np.array([0.4, 0.05, 0.18, 0.07, 0.2, 0.1]))
print("H =", entropy_alphabet)

This is very close to the entropy lower bound!

It turns out that, due to codewords being a whole number of bits, Huffman coding gives an expected length $L$ per letter
$$H(p) \leq L \leq H(p) + 1$$

### Putting it together
Let's turn this algorithm into code.

First, are there any useful datastructures for storing our alphabet as it changes?

How will we store the tree as we build it?

In [ ]:
class Node:
    def __init__(self, p, left, right=None):
        self.p = p
        self.left = left
        self.right = right
        # depth ensures the algorithm prefers to combine shallow
        # trees when selected items of equal probability
        if right is None:
            self.depth = 1
        else:
            self.depth = 1 + max(self.left.depth, self.right.depth)

    def __lt__(self, other):
        """
        Compare two tree nodes, sorting first by probability then
        by depth of tree. This is the low-level function called
        by min or the less-than operator when the arguments are
        instances of Node.
        """
        return (self.p < other.p or
                (self.p == other.p and self.depth < other.depth))
    
    def __repr__(self):
        if self.isLeaf():
            depth_or_leaf = "letter=%s" % str(self.left)
        else:
            depth_or_leaf = "depth=%d" % self.depth
        return "Node(p=%.4f, %s)" % (self.p, depth_or_leaf)
    
    def isLeaf(self):
        return self.depth == 1
    
    def walk(self, code_dict, prefix=""):
        """
        Recursive tree-walk to construct the prefix-free code by finding
        all leaves.
        """
        if self.isLeaf():
            code_dict[self.left] = prefix
        else:
            self.left.walk(code_dict, prefix + "0")
            self.right.walk(code_dict, prefix + "1")
        

def dict_to_alphabet(d):
    """
    Takes a dictionary of symbol=>probability pairs and converts them 
    to our alphabet representation for use in tree construction.
    Basically, initializes all the leaves of the tree.
    """
    pairs = [(v, k) for (k, v) in d.items()]
    nodes = [Node(p, letter) for (p, letter) in pairs]
    # Note: python compares tuples in order of the entries
    heapq.heapify(nodes)
    return nodes

In [ ]:
alphabet = dict_to_alphabet(letter_freqs)
print(alphabet)

Now, let's code up our recursive longest-subtree operation.

In [ ]:
def build_tree(alphabet):
    """
    Pop the two least-probable letters in our alphabet and create the parent/merged letter.
    Recurse until we run out of letters.
    
    Returns the root node of the tree
    """
    # Get the two smallest probability letters
    right = heapq.heappop(alphabet)
    left = heapq.heappop(alphabet)
    # Create the parent node
    parent = Node(right.p + left.p, left, right)
    # Push the parent node onto the heap (like alpha in the example)
    heapq.heappush(alphabet, parent)
    # Check if we've reached the root node, otherwise recurse
    if len(alphabet) == 1:
        return alphabet[0]
    else:
        return build_tree(alphabet)

#### Let's test it!!!

In [ ]:
example_dict = {
    "a": 0.4,
    "b": 0.05,
    "c": 0.18,
    "d": 0.07,
    "e": 0.2,
    "f": 0.1,
}

# Form the leaves
alphabet = dict_to_alphabet(example_dict)
print(alphabet)
print()

# Construct the full tree
root = build_tree(alphabet)
print(root)
print()

# Now turn the tree into a prefix-free code
encoding_dict = bidict()
root.walk(encoding_dict)
print("Code:")
for k, v in encoding_dict.items():
    print("\t%s: %s" % (k, v))

How does our code perform on English letters? Let's calculate the expected bits per letter.

In [ ]:
def encode(message, encoding):
    return "".join(encoding[l] for l in message)
    
def expected_bits_per_letter(encoding, frequencies):
    expectation = 0
    for letter, codeword in encoding.items():
        prob = frequencies[letter]
        expectation += prob * len(codeword)
    return expectation

In [ ]:
alphabet = dict_to_alphabet(letter_freqs)
root = build_tree(alphabet)
letter_encoding = bidict()
root.walk(letter_encoding)
exp_len = expected_bits_per_letter(letter_encoding, letter_freqs)
print("Expected bits per letter:", exp_len)

In [ ]:
message1_encoded = encode(message1.upper(), letter_encoding)
print("message1:", message1)
print("C(message1):", message1_encoded)
print("encoded length:", len(message1_encoded))
print("bits per letter:", len(message1_encoded) / len(message1))
print()
message2_encoded = encode(message2.upper(), letter_encoding)
print("message2:", message2)
print("C(message2):", message2_encoded)
print("encoded length:", len(message2_encoded))
print("bits per letter:", len(message2_encoded) / len(message2))


Very close to the entropy of 4.07!

We can see how messages with lots of rare letters like q, j, z take more bits to encode than messages with common letters like a, t, s.


#### Decoding
Let's make sure we can decode our encode messages.

In [ ]:
def decode(message, encoding):
    output = []
    prefix_size = 1
    while len(message) > 0:
        # Sanity check
        assert prefix_size <= len(message)
        # Check if our current prefix is a valid codeword
        maybe_cw = message[:prefix_size]
        if maybe_cw in encoding.inverse:
            # Valid: output the decoded letter, remove the prefix from the message, 
            # and reset the prefix size
            output.append(encoding.inverse[maybe_cw])
            message = message[prefix_size:]
            prefix_size = 1
        else:
            # We don't have a valid codeword yet, increase the prefix size
            prefix_size += 1
    return output

def decode_str(message, encoding):
    return "".join(decode(message, encoding))

def decode_generator(message_encoding):
    # Yielding generator form of decoder for use with jpeg123
    prefix_size = 1
    while len(message) > 0:
        # Sanity check
        assert prefix_size <= len(message)
        # Check if our current prefix is a valid codeword
        maybe_cw = message[:prefix_size]
        if maybe_cw in encoding.inverse:
            # Valid: output the decoded letter, remove the prefix from the message, 
            # and reset the prefix size
            yield encoding.inverse[maybe_cw]
            message = message[prefix_size:]
            prefix_size = 1
        else:
            # We don't have a valid codeword yet, increase the prefix size
            prefix_size += 1

In [ ]:
maybe_message1 = decode_str(message1_encoded, letter_encoding)
print("message1   :", message1)
print("message1(?):", maybe_message1)

### Message-specific Huffman codes
Can we do better for either message? What if we construct a Huffman code unique to each message?

First, we need to know the letter frequencies for each message.

In [ ]:
def calc_letter_frequencies(message):
    # Get occurence count of each letter in the message
    counts = defaultdict(lambda: 0)
    for l in message:
        counts[l] += 1
    # Convert counts to probabilities
    num_letters = len(message)
    for l in counts:
        counts[l] /= num_letters
    return dict(counts)

message1_freqs = calc_letter_frequencies(message1)
message2_freqs = calc_letter_frequencies(message2)
print(message1_freqs)
print("Unique letters:", len(message1_freqs))
print()
print(message2_freqs)
print("Unique letters:", len(message2_freqs))

 Now we can create custom Huffman codes and compare encoded lengths.

In [ ]:
alphabet = dict_to_alphabet(message1_freqs)
root = build_tree(alphabet)
letter_encoding = bidict()
root.walk(letter_encoding)
exp_len = expected_bits_per_letter(letter_encoding, message1_freqs)
message1_custom_encoded = "".join([letter_encoding[l] for l in message1])
print("Total length:", len(message1_custom_encoded))
print("Expected bits per letter:", exp_len)
print("Bits per letter:\t", len(message1_custom_encoded) / len(message1))
print("Entropy:\t\t", Hdist(np.array(list(message1_freqs.values()))))

In [ ]:
alphabet = dict_to_alphabet(message2_freqs)
root = build_tree(alphabet)
letter_encoding = bidict()
root.walk(letter_encoding)
exp_len = expected_bits_per_letter(letter_encoding, message2_freqs)
message2_custom_encoded = "".join([letter_encoding[l] for l in message2])
print("Total length:", len(message2_custom_encoded))
print("Expected bits per letter:", exp_len)
print("Bits per letter:\t", len(message2_custom_encoded) / len(message2))
print("Entropy:\t\t", Hdist(np.array(list(message2_freqs.values()))))

Much better! However, in order for a friend to decode the message they need to know what our code tree looks like. There are a few ways we can achieve this.

One option is to somehow send them our tree beforehand, like we do for the jpeg123 implementation with pre-chosen Huffman codes.

Another option is to choose a representation for the tree structure and send it along with the rest of our message. This adds overhead to our compression scheme, but if the main body of our message is much bigger than the tree representation we can still achieve good compression.

#### The tree representation is left up to you for the final project! Experiment and see how small you can make the representation while including all necessary information.

Try converting your code to a canonical Huffman code (see Wikipedia) and think about how you can serialize only the minimum information needed to reconstruct the full code.

# Custom Huffman Codes for JPEG123

Let's explore the potential compression gains from custom Huffman codes for your jpeg123 images.

In [ ]:
def get_code_dict(freq_table):
    alphabet = dict_to_alphabet(freq_table)
    root = build_tree(alphabet)
    code_dict = bidict()
    root.walk(code_dict)
    print("Expected bits/symbol:", expected_bits_per_letter(code_dict, freq_table))
    return code_dict

def get_entropy(freq_table):
    return Hdist(np.array(list(freq_table.values())))

def custom_encode_image(img, quality=75, plot_freqs=False):
    # Perform compression up to Huffman encoding
    ydc, yac, cbdc, cbac, crdc, crac = process_image(img, quality=quality)
    # Build the coefficient frequency tables
    dc_freqs = calc_letter_frequencies(ydc + cbdc + crdc)
    ac_freqs = calc_letter_frequencies(yac + cbac + crac)
    if plot_freqs:
        plt.figure(figsize=(12, 7))
        plt.bar(dc_freqs.keys(), dc_freqs.values())
        plt.title("DC coefficient probabilities")
        plt.show()
        plt.figure(figsize=(12, 7))
        plt.bar(list(map(str, ac_freqs.keys())), ac_freqs.values())
        plt.title("AC coefficient probabilities")
        plt.show()
    # Generate Huffman code dictionaries
    dc_code = get_code_dict(dc_freqs)
    ac_code = get_code_dict(ac_freqs)
    # Encode coefficients
    # DC
    ydc_c = encode(ydc, dc_code)
    cbdc_c = encode(cbdc, dc_code)
    crdc_c = encode(crdc, dc_code)
    # AC
    yac_c = encode(yac, ac_code)
    cbac_c = encode(cbac, ac_code)
    crac_c = encode(crac, ac_code)
    # Return 
    all_bits = (ydc_c, yac_c, cbdc_c, cbac_c, crdc_c, crac_c)
    return all_bits

In [ ]:
# Load image
img = Image.open("NetaLi_small.tiff")
# img = Image.open("creek.png")
# img = Image.open("glass.png")
img = np.array(img)[:,:,0:3]
M, N = img.shape[0:2]

# Display image
plt.figure(figsize=(10,10))
plt.imshow(img), plt.xticks([]), plt.yticks([])
plt.show()

# Image storage size
npixels = M * N
nbits_raw = npixels * 8 * 3
print("RGB{8,8,8} bits:", nbits_raw);

In [ ]:
# Single-byte entropy
img_rgb_freqs = calc_letter_frequencies(np.ravel(img))
img_entropy = get_entropy(img_rgb_freqs)
print("Image R/G/B entropy:", img_entropy)
print("Image bits per R/G/B value:", nbits_raw / npixels / 3)
plt.figure(figsize=(12,7))
plt.bar(img_rgb_freqs.keys(), img_rgb_freqs.values())
plt.title("R/G/B value frequencies");

In [ ]:
# RGB tuple entropy
pixels = img.reshape(-1, 3)
pixel_tuples = [(rgb[0], rgb[1], rgb[2]) for rgb in pixels]
img_pixel_freqs = calc_letter_frequencies(pixel_tuples)
img_pixel_entropy = get_entropy(img_pixel_freqs)
print("Image pixel entropy:", img_pixel_entropy)
print("Image bpp:", nbits_raw / npixels)
plt.figure(figsize=(12,7))
# plt.bar(list(map(str, img_pixel_freqs.keys())), img_pixel_freqs.values())
plt.stem(img_pixel_freqs.values())
plt.xticks([])
plt.title("Pixel value frequencies");

## Building the DC and AC codes
It turns out that the DC and AC coefficients come from sufficiently different distributions that it is often more effective to compress them with separate codes than try to include both in a single code.

Let's do that here.

In [ ]:
# Perform compression up to Huffman encoding
ydc, yac, cbdc, cbac, crdc, crac = process_image(img, quality=75)

In [ ]:
# Build the coefficient frequency tables
dc_freqs = calc_letter_frequencies(ydc + cbdc + crdc)
ac_freqs = calc_letter_frequencies(yac + cbac + crac)

Let's look at the frequency distributions.

In [ ]:
plt.figure(figsize=(12,7))
plt.bar(dc_freqs.keys(), dc_freqs.values())
plt.title("DC coefficient probabilities");

In [ ]:
plt.figure(figsize=(12,7))
plt.bar(list(map(str, ac_freqs.keys())), ac_freqs.values())
plt.title("AC coefficient probabilities");

Now we can construct the two codes.

In [ ]:
# Generate Huffman code dictionaries
dc_code = get_code_dict(dc_freqs)
ac_code = get_code_dict(ac_freqs)

What are the entropies?

In [ ]:
print("H(dc):", get_entropy(dc_freqs))
print("H(ac):", get_entropy(ac_freqs))

## Encoding the image

Remember, we want to encode each of Y, Cb, Cr and AC, DC separately so that we can recover the blocks nicely.

In [ ]:
ydc_c = encode(ydc, dc_code)
cbdc_c = encode(cbdc, dc_code)
crdc_c = encode(crdc, dc_code)

In [ ]:
print("Y DC bits:", len(ydc_c))
print("Cb DC bits:", len(cbdc_c))
print("Cr DC bits:", len(crdc_c))

In [ ]:
yac_c = encode(yac, ac_code)
cbac_c = encode(cbac, ac_code)
crac_c = encode(crac, ac_code)

In [ ]:
print("Y AC bits:", len(yac_c))
print("Cb AC bits:", len(cbac_c))
print("Cr AC bits:", len(crac_c))

In [ ]:
all_bits = (ydc_c, yac_c, cbdc_c, cbac_c, crdc_c, crac_c)
nbits_custom = sum([len(b) for b in all_bits])
print("Total bits:", nbits_custom)

Our default Huffman table in Part 1 used **76172** bits to encode all coefficients, with a compression ratio of **18.15**.

In [ ]:
nbits_default = 76172
print("Custom compression ratio:", nbits_default / nbits_custom)
print("Total compression ratio:", nbits_raw / nbits_custom)

What is the entropy of our new encoded bits? 

What do we want it to be?

In [ ]:
bit_freqs = calc_letter_frequencies("".join(all_bits))
bit_freqs

In [ ]:
Hdist(binomial(bit_freqs["1"]))

## Other quality factors

In [ ]:
# Default Huffman tables
quality = 15
bits = encode_image(img, quality=quality)
num_bits = [len(b) for b in bits]
num_pix = M * N
img_dec = decode_image(bits, M, N, quality=quality)

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(1,3,1)
plt.imshow(img), plt.xticks([]), plt.yticks([])
plt.title("Original image")
plt.subplot(1,3,2)
plt.imshow(img_dec.astype(np.uint8)), plt.xticks([]), plt.yticks([])
plt.title("Compressed image")
plt.subplot(1,3,3)
plt.imshow(abs((img_dec - img.astype(float))*10).astype(np.uint8)), plt.xticks([]), plt.yticks([])
plt.title("Error image x 10")
plt.show();

In [ ]:
custom_bits = custom_encode_image(img, quality=quality, plot_freqs=True)
nbits = sum([len(b) for b in custom_bits])
print("Extra compression ratio:", sum(num_bits) / nbits)